In [6]:
%%time
#Wall time: 3min 20seg

import pandas as pd
import numpy as np

first_year = 2016
last_year = 2020



number_of_teams_per_year = []
weekdays_total_per_year = []

for year_ in range(first_year,last_year+1):
    URL = f'https://www.resultados-futbol.com/portugal{year_}/grupo1/equipos'
    webpage_equipos = pd.read_html(URL)
    Table_year = pd.DataFrame(webpage_equipos[len(webpage_equipos)-1])
    number_of_teams_per_year.append(len(Table_year['Equipo.1']))

for number_of_teams_ in number_of_teams_per_year:
    weekdays_total_per_year.append((number_of_teams_-1)*2)

    
list_of_years_times_weekdays = []
list_of_weekdays_times_years = []

list_of_years = list(range(first_year,last_year+1))

for i in range(len(list_of_years)):
    for weekday_ in range(weekdays_total_per_year[i]):
        for number_of_games_per_weekday_ in range(int(number_of_teams_per_year[i]/2)):
            list_of_years_times_weekdays.append(list_of_years[i])
            
for i in range(len(list_of_years)):
    for weekday_ in range(1,weekdays_total_per_year[i]+1):
        for number_of_games_per_weekday_ in range(int(number_of_teams_per_year[i]/2)):
            list_of_weekdays_times_years.append(weekday_)


        
#home_teams_year_weekday, away_teams_year_weekday, only_results_year_weekday,      

home_team_list = []
away_team_list = []
results_list = []

for i in range(len(list_of_years)):
    for m in range(1,weekdays_total_per_year[i]+1):
        URL = f'https://www.resultados-futbol.com/portugal{list_of_years[i]}/grupo1/jornada{m}'

        webpage = pd.read_html(URL)
        weekday_table = pd.DataFrame(webpage[7])
        number_columns = range(weekday_table.shape[1])
        weekday_table.columns = ['To delete', 'Date', 'Home_team',
                                   'Description', 'Away_team', 'More_info']
        weekday_table = weekday_table.drop(['To delete'], axis=1)
        weekday_table = weekday_table.dropna()

        list_final_results_weekday = []

        for n in list(weekday_table['Description']):
            list_final_results_weekday.append(n[n.rindex(' ')+1:])

        weekday_table.insert(3, 'Final_Result', list_final_results_weekday, True)    

        home_team_list.append(list(weekday_table['Home_team']))
        away_team_list.append(list(weekday_table['Away_team']))
        results_list.append((weekday_table['Final_Result']))


# each one of these lists is a list of lists, and we need only one list

home_team_list_only_one_list = []
for sublist in home_team_list:
    for item in sublist:
        home_team_list_only_one_list.append(item)
        
away_team_list_only_one_list = []
for sublist in away_team_list:
    for item in sublist:
        away_team_list_only_one_list.append(item)

results_list_only_one_list = []
for sublist in results_list:
    for item in sublist:
        results_list_only_one_list.append(item)
   

#DataFrame definition

dfdata_table_results = {
          'Year': list_of_years_times_weekdays,
          'Weekday': list_of_weekdays_times_years,
          'Home_team': home_team_list_only_one_list,
          'Away_team': away_team_list_only_one_list,
          'Result': results_list_only_one_list
         }

Table_Results_web_scrapping = pd.DataFrame(dfdata_table_results)

#DataFrame enrichment

list_all_scores = list(Table_Results_web_scrapping['Result'])
list_home_score = []
list_away_score = []

for score in list_all_scores:
    list_home_score.append(score[:score.index('-')])
    list_away_score.append(score[score.index('-')+1:])

Table_Results_web_scrapping['Home_score'] = list_home_score
Table_Results_web_scrapping['Away_score'] = list_away_score
Table_Results_web_scrapping.loc[Table_Results_web_scrapping['Home_score'] == Table_Results_web_scrapping['Away_score'], '1x2'] = 'x'
Table_Results_web_scrapping.loc[Table_Results_web_scrapping['Home_score'] > Table_Results_web_scrapping['Away_score'], '1x2'] = '1'
Table_Results_web_scrapping.loc[Table_Results_web_scrapping['Home_score'] < Table_Results_web_scrapping['Away_score'], '1x2'] = '2'
Table_Results_web_scrapping.loc[Table_Results_web_scrapping['1x2'] == 'x', 'Points_Home_Team'] = int('1')
Table_Results_web_scrapping.loc[Table_Results_web_scrapping['1x2'] == '1', 'Points_Home_Team'] = int('3')
Table_Results_web_scrapping.loc[Table_Results_web_scrapping['1x2'] == '2', 'Points_Home_Team'] = int('0')
Table_Results_web_scrapping.loc[Table_Results_web_scrapping['1x2'] == 'x', 'Points_Away_Team'] = int('1')
Table_Results_web_scrapping.loc[Table_Results_web_scrapping['1x2'] == '1', 'Points_Away_Team'] = int('0')
Table_Results_web_scrapping.loc[Table_Results_web_scrapping['1x2'] == '2', 'Points_Away_Team'] = int('3')

Table_Results_web_scrapping

CPU times: user 41 s, sys: 908 ms, total: 41.9 s
Wall time: 2min 34s


,Year,Weekday,Home_team,Away_team,Result,Home_score,Away_score,1x2,Points_Home_Team,Points_Away_Team
0,2016,1,Tondela,Sporting CP,1-2,1,2,2,0.0,3.0
1,2016,1,Os Belenenses,Rio Ave,3-3,3,3,x,1.0,1.0
2,2016,1,Porto,Vitória Guimarães,3-0,3,0,1,3.0,0.0
3,2016,1,Vitória Setúbal,Boavista,2-2,2,2,x,1.0,1.0
4,2016,1,União Madeira,Marítimo,2-1,2,1,1,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...
1525,2020,34,Benfica,Sporting CP,2-1,2,1,1,3.0,0.0
1526,2020,34,Sporting Braga,Porto,2-1,2,1,1,3.0,0.0
1527,2020,34,Moreirense,Tondela,1-2,1,2,2,0.0,3.0
1528,2020,34,Portimonense,Desportivo Aves,2-0,2,0,1,3.0,0.0


In [15]:
URL = f'https://www.resultados-futbol.com/portugal2013/grupo1/jornada3'

webpage = pd.read_html(URL)
weekday_table = pd.DataFrame(webpage[7])
number_columns = range(weekday_table.shape[1])
weekday_table.columns = ['To delete', 'Date', 'Home_team',
                           'Description', 'Away_team', 'More_info']
weekday_table = weekday_table.drop(['To delete'], axis=1)
weekday_table = weekday_table.dropna()

for m in list(weekday_table['Description']):
    print(m)

Vitória Guimarães - Estoril  2012-09-01T19:30:00  Estádio Dom Afonso Henriques  2-2
Olhanense - Porto  2012-09-01T21:30:00  Estádio José Arcanjo  2-3
Beira Mar SC - Moreirense  2012-09-02T17:00:00  Estádio Municipal de Aveiro  1-1
Gil Vicente - Vitória Setúbal  2012-09-02T17:00:00  Estádio Cidade de Barcelos  0-0
Rio Ave - Académica  2012-09-02T19:00:00  Estádio do Rio Ave Futebol Clube  0-0
Paços de Ferreira - Sporting Braga  2012-09-02T19:00:00  Estádio da Capital do Móvel  2-0
Benfica - Nacional  2012-09-02T21:15:00  Estádio da Luz  3-0
Marítimo - Sporting CP  2012-09-16T21:15:00  Estádio dos Barreiros  (0) 1-1 (0)


<div class  = "alert alert-info"> 
** ATTENTION **<br>
There is a problem is with the last game of the year 2013, weekday 3: Marítimo vs Sporting.<br>
Exceptionally, the result of this game is not after the last space, as it is for all other matches.<br>
See above.
</div>

In [5]:
pip install openpyxl

     |████████████████████████████████| 243 kB 3.6 MB/s eta 0:00:01
You should consider upgrading via the '/opt/anaconda3/envs/ksunsupervised/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [7]:
Table_Results_web_scrapping.to_excel('Table_results.xlsx')

In [8]:
Table_Results = pd.read_excel('Table_results.xlsx')
Table_Results.drop(['Unnamed: 0'],axis = 1, inplace = True)

In [9]:
Table_Results

,Year,Weekday,Home_team,Away_team,Result,Home_score,Away_score,1x2,Points_Home_Team,Points_Away_Team
0,2016,1,Tondela,Sporting CP,1-2,1,2,2,0,3
1,2016,1,Os Belenenses,Rio Ave,3-3,3,3,x,1,1
2,2016,1,Porto,Vitória Guimarães,3-0,3,0,1,3,0
3,2016,1,Vitória Setúbal,Boavista,2-2,2,2,x,1,1
4,2016,1,União Madeira,Marítimo,2-1,2,1,1,3,0
...,...,...,...,...,...,...,...,...,...,...
1525,2020,34,Benfica,Sporting CP,2-1,2,1,1,3,0
1526,2020,34,Sporting Braga,Porto,2-1,2,1,1,3,0
1527,2020,34,Moreirense,Tondela,1-2,1,2,2,0,3
1528,2020,34,Portimonense,Desportivo Aves,2-0,2,0,1,3,0
